# PLEQUE  for COMPASS-U Fiesta files
PLEQUE = PLasma EQUilibrium Enjoyment module

#### Important imports:

In [ ]:
%pylab notebook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy.ma as ma

In [ ]:
from pleque.io import compass
from pleque.utils import plotting
import pleque.tests.utils as test_util

In [ ]:
test_util.get_test_cases_number()

Let's load COMPASS testing shot and plot its overview. 

Other posibilities how to load equilibrium are showed at https://pleque.readthedocs.io/en/

In [ ]:
eq = test_util.load_testing_equilibrium(4)
plt.figure()
eq.plot_overview()

## Get basic information about plasma is simple:

In [ ]:
# generate simple grid for printing (instance of Coordinates class) 
grid = eq.grid((200,300), dim='size')

# R, Z coordinates of the grid: 
R = grid.R
Z = grid.Z

# poloidal flux: 
psi = grid.psi 

# or from equilibrium: 
psi = eq.psi(grid)

# Last closed flux surface (LCFS))
# lcfs is set of coordinates points (class Coordinates) so we choose one of them 
lcfs = eq.lcfs

# psi of last closed fluxsurface (psi of one point laying on lcfs):
psi_lcfs = lcfs.psi[0]
# rs_lcfs = lcfs.R, zs_lcfs = lcfs.Z, ... 

# first wall
first_wall = eq.first_wall

# The absolute value of the poloidal magnetic field: 
B_pol = eq.B_pol(grid)

# Toroidal magnetic field
B_tor = eq.B_tor(grid)

# Toroidal current density: 
j_tor = eq.j_tor(grid)

# the current is calculated from psi map and profiles 
# let's put mask of separatrix on j_tor 
j_tor_ma = ma.masked_array(j_tor, np.logical_not(eq.in_lcfs(grid)))

# etc... (use tab completion eq.[TAB])


### Quantities plotting: 

In [ ]:
# generate simple grid for printing (instance of Coordinates class) 

fig, axs = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(10,4))

# Fast decoration of the plot
def decorate_ax(ax: plt.Axes):
    ax.plot(lcfs.R, lcfs.Z, color='k', ls='--', lw=2)
    ax.plot(first_wall.R, first_wall.Z, 'k-', lw=2)
    ax.set_xlabel('R [m]')
    ax.set_ylabel('Z [m]')
    ax.set_aspect('equal')

# lets' plot psi
ax = axs[0]
ax.set_title(r'$\psi$')
# psi is one of the coordinates so it can be evaluate as grid.psi, 
# it is equivalent to eq.psi(grid)
ax.contour(grid.R, grid.Z, grid.psi, 20)
# plot lcfs and first wall (limiter)
ax.plot(lcfs.R, lcfs.Z, color='k', ls='--', lw=2)
ax.contour(grid.R, grid.Z, grid.psi, [eq.lcfs.psi[0]], colors='C1')
ax.plot(first_wall.R, first_wall.Z, 'k-', lw=2)

# some decoration: 
ax.set_xlabel('R [m]')
ax.set_ylabel('Z [m]')
ax.set_aspect('equal')

# Now poloidal field: 
ax = axs[1]
ax.set_title(r'$|B_\theta|$')
# Magnetic field is not coordinate, so it must be called from Equilibrium class:
cs = ax.contour(grid.R, grid.Z, B_pol, np.linspace(0, 0.6, 9))
plt.clabel(cs, inline=1)
decorate_ax(ax)

# Toroidal field plot: 
ax = axs[2]
ax.set_title(r'$|B_\phi|$')
cs = ax.contour(grid.R, grid.Z, B_tor, 17)
plt.clabel(cs, inline=1)
decorate_ax(ax)

# Tor current density: 
ax = axs[3]
ax.set_title(r'$|j_\phi|$')
cs = ax.contour(grid.R, grid.Z, j_tor_ma, 6)
plt.clabel(cs, inline=1)
decorate_ax(ax)



## Now some equilibrium midplane profiles:
As a first step we have to generate some midplane coordinates. 
We have multiple options for this. Let's choose the fastest way how to generete them. 
More options to generate coordinates will be shown later in the notebook. 


In [ ]:
# we will generate coordinates in polar system with it's center on the magnetic axis:
# (!) there is lower case r, not capitalized R which denotes to cylindrical coordinates

# number of points
N = 100

# this is not the simpliest way... It will be more handy in the near future
# we find the minor radius on theta = 0)
r_sep = eq.lcfs.r[np.argmin(np.abs(eq.lcfs.theta))]

# and here is the future (mapping to outer midplane) 
r_sep = eq.lcfs.r_mid[0]

# module automaticaly identify the type of the input: 
# in this case r is minor radius and theta poloidal angle (align with the magnetic axis)
# by saying grid=False we don't generate grid data
midplane = eq.coordinates(r=np.linspace(0, r_sep, N), theta=np.zeros(N), grid=False)

# get the midplane profiles: 
B_tor = eq.B_tor(midplane)
B_pol = eq.B_pol(midplane)
B_abs = eq.B_abs(midplane)

# toroidal/poloidal flux: 
tor_flux = eq.tor_flux(midplane)
pol_flux = eq.psi(midplane)

# on the first call q profile is calculated 
# it is (not precise for psi_n -> 1)
# for more precise method see bellow
q_profile = eq.q(midplane)

fig, axs = plt.subplots(3, 1, sharex=True)


# First profiles of magnetic fields on midplane: 
ax = axs[0]
# Profile of toroidal field: 
ax.plot(midplane.r, eq.B_tor(midplane), label=r'$B_\phi$')
# Profile of poloidal field: 
ax.plot(midplane.r, eq.B_pol(midplane), label=r'$B_\theta$')
# total
ax.plot(midplane.r, eq.B_abs(midplane), label=r'$|\vec B|$')
ax.axvline(r_sep, color='k', ls='--')
ax.set_ylabel(r'$B$ [T]')
ax.legend()


# Field coordinates (poloidal and toroidal flux)
ax = axs[1]
ax2 = ax.twinx()
# ax.plot(midplane.r, eq.tor_flux(r=midplane.r))
ax.plot(midplane.r, eq.tor_flux(midplane), 'C0', label='Toroidal mg. flux')
ax.axvline(r_sep, color='k', ls='--')
ax.tick_params(labelcolor='C0')
ax.set_ylabel(r'$\Phi$')
ax.legend()

ax2.plot(midplane.r, midplane.psi, 'C1', label='Poloidal mg. flux')
ax2.tick_params(labelcolor='C1')
ax2.set_ylabel(r'$\psi$')

ax2.legend()

# q-profile
ax = axs[2]
ax.plot(midplane.r, np.abs(eq.q(midplane)), label='q')
ax.set_ylabel(r'$q$')
ax.axvline(r_sep, color='k', ls='--')
ax.set_ylim([0, 5])

ax.set_xlabel(r'$r_\mathrm{mid}$')


plt.tight_layout()

# ax = gca()
# ax.set_title(r'$\psi$')
# psi is one of the coordinates so it can be evaluate as grid.psi, 
# it is equivalent to eq.psi(grid)
# ax.contour(grid.R, grid.Z, grid.psi, 20)
# decorate_ax(ax)


## Spline interpolation of fitted data and mapping to various coordinates

In [ ]:
from scipy.special import erf


N = 200

chord1 = eq.coordinates(R=0.6*np.ones(N), Z=np.linspace(0.3, 0., N))
chord2 = eq.coordinates(R=np.linspace(0.35, 0.6, 20), Z=-0.1*np.ones(20))


# let's define some testing profile function: 
prof_func = lambda x, k1, xsep: k1/4 * (1 + erf((x-xsep)*20))*np.log((x+1)*1.2) - 4*np.exp(-(50*(x-1)**2))
chord_prof = prof_func(1 - chord1.psi_n, 10, 0.15)

# todo here
plt.figure()
plt.plot(chord1.Z, chord_prof, color='C3')
eq.fluxfuncs.add_flux_func('test_prof', chord_prof, chord1, spline_smooth=0)
plt.plot(chord1.Z, eq.fluxfuncs.test_prof(chord1), '--', color='C3')
plt.xlabel('Z [m]')
plt.ylabel('profile value [a.u.]')
plt.tight_layout()

plt.figure()
plt.plot(chord2.R, eq.fluxfuncs.test_prof(chord2), '--', color='C4')
plt.xlabel('R [m]')
plt.ylabel('profile value [a.u.]')
plt.tight_layout()


plt.figure()
plt.plot(chord1.r_mid, chord_prof, color='C1')
plt.xlabel(r'$r_\mathrm{mid}$ [m]')
plt.ylabel('profile value [a.u.]')

plt.tight_layout()

plt.figure()
eq._plot_overview()

chord1.plot(lw=3, ls='--', color='C3')
chord2.plot(lw=3, ls='--', color='C4')




In [ ]:
from scipy.special import erf

N = 200

chord1 = eq.coordinates(R=0.9*np.ones(N), Z=np.linspace(0.4, -0.1, N))
chord2 = eq.coordinates(R=np.linspace(0.65, 0.9, 20), Z=-0.1*np.ones(20))


# let's define some testing profile function: 
prof_func = lambda x, k1, xsep: k1/4 * (1 + erf((x-xsep)*20))*np.log((x+1)*5)
chord_prof = prof_func(1 - chord1.psi_n, 15, 0.1)

# todo here
plt.figure()
plt.plot(chord1.Z, chord_prof)
eq.fluxfuncs.add_flux_func('test_prof', chord_prof, chord1, spline_smooth=1)

plt.figure()
eq._plot_overview()




## Test of 3D coordinates: 

In [ ]:
# Let's define some detector: 
# R = 1.25, Z = 0, phi = 0 pointing to (direction)
# x = -1, y = 0.5, z = 0

direction = np.array((-1, 0.6, 0.2))
direction /= np.linalg.norm(direction)
position = np.array((1.25, 0, -0.1))

# Let's define limiters: 
Ns = 100
inner_lim = eq.coordinates(np.min(eq.first_wall.R)*np.ones(Ns), np.zeros(Ns), np.linspace(0, 2*np.pi, Ns))
outer_lim = eq.coordinates(np.max(eq.first_wall.R)*np.ones(Ns), np.zeros(Ns), np.linspace(0, 2*np.pi, Ns))

# Line of view of the detector (i.e. camera pixel):
camera_view = eq.coordinates(position+direction[np.newaxis,:]*np.linspace(0, 2.0, 20)[:, np.newaxis], coord_type=('X', 'Y', 'Z'))

fig, axs = plt.subplots(1,2)
# Upper view:
ax = axs[0]
ax.plot(inner_lim.X, inner_lim.Y, 'k-')
ax.plot(outer_lim.X, outer_lim.Y, 'k-')
ax.plot(camera_view.X, camera_view.Y, 'x--', label='Chord of the view')
ax.plot(position[0], position[1], 'd', color='C0')
ax.legend()
ax.set_aspect('equal')

ax = axs[1]
ax.plot(eq.first_wall.R, eq.first_wall.Z, 'k-')
ax.plot(eq.lcfs.R, eq.lcfs.Z, 'k--')
ax.plot(camera_view.R, camera_view.Z, 'x--')
ax.plot(position[0], position[2], 'd', color='C0')
ax.set_aspect('equal')

In [ ]:
ax.contour(grid.R, grid.Z, grid.psi, [eq.lcfs.psi[0]], colors='C1')